# atmodeller

## Silicon

Import the required functionality.

In [ ]:
from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import SystemConstraints, ElementMassConstraint, BufferedFugacityConstraint, TotalPressureConstraint, ActivityConstraint
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer, IronWustiteBufferBallhaus, IronWustiteBufferFischer, IronWustiteBufferHirschmann21, IronWustiteBufferHirschmann
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet
from atmodeller.initial_solution import InitialSolutionDict, InitialSolutionLast
#InitialSolution, InitialSolutionRegressor, InitialSolutionSwitchRegressor, InitialSolutionConstant
from atmodeller.core import GasSpecies, Species, LiquidSpecies
from atmodeller.solubility.hydrogen_species import H2_basalt_hirschmann, H2O_peridotite_sossi
from atmodeller.utilities import earth_oceans_to_kg
from atmodeller.eos.interfaces import RealGas
from atmodeller.eos.saxena import get_saxena_eos_models, H2_SS92
from atmodeller.eos.holland import get_holland_eos_models, H2_CORK_HP91, H2O_CORK_HP98
from atmodeller.eos.holley import get_holley_eos_models, H2_Beattie_holley
from atmodeller.eos.chabrier import get_chabrier_eos_models, H2_CD21

import logging
# logger = debug_file_logger()
logger = debug_logger()
logger.setLevel(logging.INFO)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

model_holland = H2_CORK_HP91
model_holley = H2_Beattie_holley
model_saxena = H2_SS92
model_chabrier = H2_CD21

## 1A. Compare H2 fugacity coefficients

In [ ]:
pressures = np.logspace(0, 6, num=200)
temperature = 3000 #k


fc_holland = []
fc_holley = []
fc_saxena = []
fc_chabrier = []

for pressure in pressures:
    fc_holland.append(model_holland.fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_holley.append(model_holley.fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_saxena.append(model_saxena.fugacity_coefficient(temperature=temperature, pressure=pressure))
    fc_chabrier.append(model_chabrier.fugacity_coefficient(temperature=temperature, pressure=pressure))

fig, ax1 = plt.subplots(1, figsize=(4,4), tight_layout='True')

ax1.plot(pressures/1e3, fc_holland, color='orange', lw=2, label=r'Holland')
ax1.plot(pressures/1e3, fc_holley, color='blue', lw=2, label=r'Holley')
ax1.plot(pressures/1e3, fc_saxena, color='red', lw=2, label=r'Saxena')
ax1.plot(pressures/1e3, fc_chabrier, color='green', lw=2, label=r'Chabrier')

ax1.set_title('H2 fugacity coefficients at %d K'%temperature)

ax1.set_ylim([0.1,1e10])

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Total Pressure [kbar]', fontsize=14)
ax1.set_ylabel(r'H2 Fugacity Coefficient', fontsize=14)

# ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
# ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)

# ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=1, loc='upper left')
plt.savefig('H2fugcoeff_%dK.pdf'%temperature, bbox_inches='tight')
plt.savefig('H2fugcoeff_%dK.png'%temperature, bbox_inches='tight')
plt.show()

## 1B. Compare fO2 buffers

In [ ]:
pressures = np.logspace(0, 6, num=200)
temperature = 3400 #k

log10fO2_hirschmann21 = np.array([IronWustiteBufferHirschmann21.get_log10_value(IronWustiteBufferHirschmann21(), temperature, pressure) for pressure in pressures])
log10fO2_hirschmann = np.array([IronWustiteBufferHirschmann.get_log10_value(IronWustiteBufferHirschmann(), temperature, pressure) for pressure in pressures])
log10fO2_ballhaus = np.array([IronWustiteBufferBallhaus.get_log10_value(IronWustiteBufferBallhaus(), temperature, pressure) for pressure in pressures])
log10fO2_fischer = np.array([IronWustiteBufferFischer.get_log10_value(IronWustiteBufferFischer(), temperature, pressure) for pressure in pressures])

fig, ax1 = plt.subplots(1, figsize=(4,4), tight_layout='True')

ax1.plot(pressures/1e3, log10fO2_hirschmann21, color='orange', lw=2, label=r'Hirschmann et al. (2021)')
ax1.plot(pressures/1e3, log10fO2_hirschmann, color='blue', lw=2, label=r'Hirschmann et al. (2008)')
ax1.plot(pressures/1e3, log10fO2_ballhaus, color='red', lw=2, label=r'Ballhaus et al. (1991)')
ax1.plot(pressures/1e3, log10fO2_fischer, color='green', lw=2, label=r'Fischer et al. (2011)')

ax1.set_title(r'log$_{10} f_{O_2}$ at %d K'%temperature)

#ax1.set_ylim([0.1,1e10])

ax1.set_xscale('log')
#ax1.set_yscale('log')
ax1.set_xlabel(r'Total Pressure [kbar]', fontsize=14)
ax1.set_ylabel(r'log$_{10} f_{O_2}$ ', fontsize=14)

# ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
# ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)

# ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=1, loc='upper left')
plt.savefig('log10fO2_%dK.pdf'%temperature, bbox_inches='tight')
plt.savefig('log10fO2_%dK.png'%temperature, bbox_inches='tight')
plt.show()

## 2A. Example with Element Mass Constraint for Si

In [ ]:
logger: logging.Logger = debug_file_logger()

surface_temperature=3000 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

massH_ocean = 1800 # mass of H equivalent to water oceans
h_kg: float = earth_oceans_to_kg(massH_ocean)
si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
O2Si_g = GasSpecies("O2Si")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_g, O2Si_l])

constraint: SystemConstraints = SystemConstraints(
    [
        BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=-1)),
        ElementMassConstraint("H", value=h_kg),
        ElementMassConstraint("Si", value=si_kg),
#        ActivityConstraint(O2Si_l, 1),
    ]
)

system = InteriorAtmosphereSystem(species=species, planet=planet)

system.solve(constraint, tol=1e-5)

print(system.output)

#print(system.output['H2O_g'][0]['melt_moles'])

ideal_solution = system.solution.solution_dict()

logger.info("Ideal solution = %s", ideal_solution)

## 2B. Example with Element Mass Constraint for Si and Mg

In [ ]:
logger: logging.Logger = debug_file_logger()

surface_temperature=3400 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

h_kg: float = 0.01 * planet_mass 
si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
O2Si_g = GasSpecies("O2Si")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
OMg_l = LiquidSpecies("OMg")
Mg_g = GasSpecies("Mg")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_g, O2Si_l, OMg_l, Mg_g])

constraint: SystemConstraints = SystemConstraints(
    [
        BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=-4)),
        ElementMassConstraint("H", value=h_kg),
        ElementMassConstraint("Si", value=si_kg),
        ElementMassConstraint("Mg", value=si_kg),
    ]
)

system = InteriorAtmosphereSystem(species=species, planet=planet)

system.solve(constraint)

print(system.output)

print(system.output['H2O_g'][0]['melt_moles'])

ideal_solution = system.solution.solution_dict()

logger.info("Ideal solution = %s", ideal_solution)

## 2C. InitialSolutionLast implementation

In [ ]:
surface_temperature=3000 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

log_fO2_dIWs = np.linspace(-6, 2, num=200) 

massH_ocean = 1800 # mass of H equivalent to water oceans
h_kg: float = earth_oceans_to_kg(massH_ocean)

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_l])

first_solution = {
    H2_g: 6133.037295534539,
    H2O_g: 69.82341826343868,
    O2_g: 6.09059364343756e-05,
    OSi_g: 342.1036405652872,
    H4Si_g: 163971.5617881446,
    O2Si_l: 0.9999999999999993
    }

######## Dan to check here ######## 
initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )
    
    ######## Dan to check here ######## 
    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, max_attempts=10, factor=0.10, min_log10_pressure=-50)
    #initial_solution_first = initial_solution

    solution = system.solution.solution_dict()

    solution['total_pressure'] = system.atmosphere_pressure
    solution['log_fO2_dIW'] = log_fO2_dIW
    try:  
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

#data = system.output("SiOH_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

## 3A. H-O-Si system (ideal vs real) - parameter: oxygen fugacity 

### Set basic parameters

In [ ]:
surface_temperature=3000 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

massH_ocean_previous = 800
massH_ocean = 1000 # mass of H equivalent to water oceans
h_kg_previous: float = earth_oceans_to_kg(massH_ocean_previous)
h_kg: float = earth_oceans_to_kg(massH_ocean)

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

log_fO2_dIWs = np.linspace(-6, 2, num=200)


### Calculate ideal with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_3000K_800ocean.csv"
#filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean_previous)

df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
#initial_solution_first = InitialSolutionDict(first_solution, species=species)
#initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate ideal with solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# ideal with solubility
H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann())
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )
    
        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue


filename = "SiOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_solubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate real with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Non-ideal H2 and H2O
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"])
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"])        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue


filename = "SiOH_nosolubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_real_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate real with solubility solutions

In [ ]:
filename = "SiOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Real + solubility 
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"], solubility=H2_basalt_hirschmann())
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"], solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )
        
        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_solubility_real_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Plot mixing ratios: ideal vs real vs real with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiOH_nosolubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_real = pd.read_csv(filename) 
ratio_real = df_real['H2_g'].values / df_real['H2O_g'].values
logfO2_real = df_real['log_fO2_dIW'].values
H2_real = df_real['H2_g'].values
H2O_real = df_real['H2O_g'].values
SiH4_real = df_real['H4Si_g'].values
SiO_real = df_real['OSi_g'].values
O2_real = df_real['O2_g'].values
tot_real = df_real['total_pressure'].values
fcH2_real = df_real['fugacity_coefficient_H2'].values
fcH2O_real = df_real['fugacity_coefficient_H2O'].values
Hmelt_real = df_real['melt_fraction_H'].values
log_fO2_dIWs_real = df_real['log_fO2_dIW'].values

filename = "SiOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -2, alpha=0.2, color='green')
ax1.axvspan(-2, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -3.9, alpha=0.2, color='red')
ax2.axvspan(-3.9, -1.1, alpha=0.2, color='green')
ax2.axvspan(-1.1, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -5.1, alpha=0.2, color='red')
ax3.axvspan(-5.1, -0.4, alpha=0.2, color='green')
ax3.axvspan(-0.4, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-4.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/tot_ideal, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/tot_ideal, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/tot_ideal, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/tot_ideal, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1.2])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)


ax2.plot(log_fO2_dIWs_real, H2_real/tot_real, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, H2O_real/tot_real, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, SiH4_real/tot_real, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, SiO_real/tot_real, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atmospheric Composition (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'real + no solubility') 
ax2.set_ylim([1e-6, 1.2])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Mixing Ratios', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/tot_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/tot_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/tot_realsol, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/tot_realsol, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'real + solubility') 
ax3.set_ylim([1e-6, 1.2])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Mixing Ratios', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal (no solubility)')
ax4.plot(log_fO2_dIWs_real, tot_real/1000, lw=2, ls= '-.', color='black', label=r'real (no solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'real (solubility)')

ax4.set_ylim([1e0, 1e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_real_realsol_%dK_%docean.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiOH_ideal_real_realsol_%dK_%docean.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### Plot partial pressures: ideal vs real vs real with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiOH_nosolubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_real = pd.read_csv(filename) 
ratio_real = df_real['H2_g'].values / df_real['H2O_g'].values
logfO2_real = df_real['log_fO2_dIW'].values
H2_real = df_real['H2_g'].values
H2O_real = df_real['H2O_g'].values
SiH4_real = df_real['H4Si_g'].values
SiO_real = df_real['OSi_g'].values
O2_real = df_real['O2_g'].values
tot_real = df_real['total_pressure'].values
fcH2_real = df_real['fugacity_coefficient_H2'].values
fcH2O_real = df_real['fugacity_coefficient_H2O'].values
Hmelt_real = df_real['melt_fraction_H'].values
log_fO2_dIWs_real = df_real['log_fO2_dIW'].values

filename = "SiOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -2, alpha=0.2, color='green')
ax1.axvspan(-2, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -3.9, alpha=0.2, color='red')
ax2.axvspan(-3.9, -1.1, alpha=0.2, color='green')
ax2.axvspan(-1.1, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -5.1, alpha=0.2, color='red')
ax3.axvspan(-5.1, -0.4, alpha=0.2, color='green')
ax3.axvspan(-0.4, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-4.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/1e3, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/1e3, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/1e3, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/1e3, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1e3])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)


ax2.plot(log_fO2_dIWs_real, H2_real/1e3, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, H2O_real/1e3, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, SiH4_real/1e3, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_real, SiO_real/1e3, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atmospheric Composition (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'real + no solubility') 
ax2.set_ylim([1e-6, 1e3])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/1e3, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/1e3, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/1e3, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/1e3, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'real + solubility') 
ax3.set_ylim([1e-6, 1e3])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1e3, lw=2, ls= '--', color='black', label=r'Ideal (no solubility)')
ax4.plot(log_fO2_dIWs_real, tot_real/1e3, lw=2, ls= '-.', color='black', label=r'real (no solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1e3, lw=2, ls= '-', color='black', label=r'real (solubility)')

ax4.set_ylim([1e0, 1e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_real_realsol_%dK_%docean_pressures.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiOH_ideal_real_realsol_%dK_%docean_pressures.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### Plot mixing ratios: ideal vs ideal with solubility vs real with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_idealsol = pd.read_csv(filename) 
ratio_idealsol = df_idealsol['H2_g'].values / df_idealsol['H2O_g'].values
logfO2_idealsol = df_idealsol['log_fO2_dIW'].values
H2_idealsol = df_idealsol['H2_g'].values
H2O_idealsol = df_idealsol['H2O_g'].values
SiH4_idealsol = df_idealsol['H4Si_g'].values
SiO_idealsol = df_idealsol['OSi_g'].values
O2_idealsol = df_idealsol['O2_g'].values
tot_idealsol = df_idealsol['total_pressure'].values
fcH2_idealsol = df_idealsol['fugacity_coefficient_H2'].values
fcH2O_idealsol = df_idealsol['fugacity_coefficient_H2O'].values
Hmelt_idealsol = df_idealsol['melt_fraction_H'].values
log_fO2_dIWs_idealsol = df_idealsol['log_fO2_dIW'].values

filename = "SiOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -1.8, alpha=0.2, color='green')
ax1.axvspan(-1.8, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -5.6, alpha=0.2, color='red')
ax2.axvspan(-5.6, -0.2, alpha=0.2, color='green')
ax2.axvspan(-0.2, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -4.3, alpha=0.2, color='red')
ax3.axvspan(-4.3, -0.6, alpha=0.2, color='green')
ax3.axvspan(-0.6, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/tot_ideal, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/tot_ideal, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/tot_ideal, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/tot_ideal, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1.2])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)


ax2.plot(log_fO2_dIWs_idealsol, H2_idealsol/tot_idealsol, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, H2O_idealsol/tot_idealsol, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiH4_idealsol/tot_idealsol, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiO_idealsol/tot_idealsol, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'Ideal + solubility') 
ax2.set_ylim([1e-6, 1.2])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Mixing Ratios', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/tot_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/tot_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/tot_realsol, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/tot_realsol, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-6, 1.2])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Mixing Ratios', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(log_fO2_dIWs_idealsol, tot_idealsol/1000, lw=2, ls= '-.', color='black', label=r'Ideal gas (solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e0, 2e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_idealsol_realsol_%dK_%docean.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiOH_ideal_idealsol_realsol_%dK_%docean.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### Plot partial pressures: ideal vs ideal with solubility vs real with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_idealsol = pd.read_csv(filename) 
ratio_idealsol = df_idealsol['H2_g'].values / df_idealsol['H2O_g'].values
logfO2_idealsol = df_idealsol['log_fO2_dIW'].values
H2_idealsol = df_idealsol['H2_g'].values
H2O_idealsol = df_idealsol['H2O_g'].values
SiH4_idealsol = df_idealsol['H4Si_g'].values
SiO_idealsol = df_idealsol['OSi_g'].values
O2_idealsol = df_idealsol['O2_g'].values
tot_idealsol = df_idealsol['total_pressure'].values
fcH2_idealsol = df_idealsol['fugacity_coefficient_H2'].values
fcH2O_idealsol = df_idealsol['fugacity_coefficient_H2O'].values
Hmelt_idealsol = df_idealsol['melt_fraction_H'].values
log_fO2_dIWs_idealsol = df_idealsol['log_fO2_dIW'].values

filename = "SiOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -1.8, alpha=0.2, color='green')
ax1.axvspan(-1.8, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -5.6, alpha=0.2, color='red')
ax2.axvspan(-5.6, -0.2, alpha=0.2, color='green')
ax2.axvspan(-0.2, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -4.3, alpha=0.2, color='red')
ax3.axvspan(-4.3, -0.6, alpha=0.2, color='green')
ax3.axvspan(-0.6, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/1e3, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/1e3, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/1e3, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/1e3, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-8, 2e3])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)


ax2.plot(log_fO2_dIWs_idealsol, H2_idealsol/1e3, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, H2O_idealsol/1e3, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiH4_idealsol/1e3, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiO_idealsol/1e3, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'Ideal + solubility') 
ax2.set_ylim([1e-8, 2e3])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/1e3, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/1e3, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/1e3, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/1e3, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-8, 2e3])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(log_fO2_dIWs_idealsol, tot_idealsol/1000, lw=2, ls= '-.', color='black', label=r'Ideal gas (solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e0, 2e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_idealsol_realsol_%dK_%docean_pressures.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiOH_ideal_idealsol_realsol_%dK_%docean_pressures.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### Fugacity coefficients

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,3), tight_layout='True')

ax1.plot(log_fO2_dIWs_ideal, fcH2_ideal, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, fcH2O_ideal, color='blue', lw=2, ls= '--')
# ax1.plot(log_fO2_dIWs, fcH2_real, color='orange', lw=2, ls= '-.')
# ax1.plot(log_fO2_dIWs, fcH2O_real, color='blue', lw=2, ls= '-.')
l1, = ax1.plot(log_fO2_dIWs_realsol, fcH2_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
l2, = ax1.plot(log_fO2_dIWs_realsol, fcH2O_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')

ax1.set_title(r'Fugacity Coefficients') #: $T_{\rm surface}$ = %d K'%surface_temperature)

ax1.set_ylim([8e-1, 2e2])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Fugacity Coefficients', fontsize=12)

ax1.legend()
old_handles, labels = ax1.get_legend_handles_labels()

custom_handles = [Line2D([0], [0], color='black', ls='--', lw=2, label='Ideal gas'),
#                  Line2D([0], [0], color='black', ls='-.', lw=2, label='Nondeal (no solubility)'),
                  Line2D([0], [0], color='black', ls='-', lw=2, label='Real gas')]

ax1.legend(handles=custom_handles + old_handles, ncol=2, loc='upper right')

plt.savefig("fugacitycoeff_SiOH_real_realsol_%dK_%docean.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("fugacitycoeff_SiOH_real_realsol_%dK_%docean.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### H partitioning

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(6,3), tight_layout='True')

ax1.plot(log_fO2_dIWs_ideal, 100 * Hmelt_ideal, color='black', lw=2, ls= '--',  label='Ideal gas (no solubility)')
ax1.plot(log_fO2_dIWs_idealsol, 100 * Hmelt_idealsol, color='black', lw=2, ls= '-.', label='Ideal gas (solubility)')
ax1.plot(log_fO2_dIWs_realsol, 100 * Hmelt_realsol, color='black', lw=2, ls= '-', label='Real gas (solubility)')

ax1.set_title(r'Hydrogen Partitioning in Magma') #: $T_{\rm surface}$ = %d K'%surface_temperature)

#ax1.set_ylim([8e-1, 3e1])
#ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'H fraction in magma [%]', fontsize=12)

ax1.legend(ncol=1, loc='lower right')

plt.savefig("Hmelt_SiOH_real_realsol_%dK_%docean.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("Hmelt_SiOH_real_realsol_%dK_%docean.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

# 3B. H-O-Si-Mg 

### Set basic params

In [ ]:
surface_temperature=3400 # kelvin
surface_temperature_previous=3000
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

massH_ocean = 1000 # mass of H equivalent to water oceans
h_kg: float = earth_oceans_to_kg(massH_ocean)

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

log_fO2_dIWs = np.linspace(-6, 2, num=200) 

### Calculate ideal with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature_previous,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
#df['H2'] = df['H2'] * adjust_sol(df['H2']) 
#df['H4Si'] = df['H4Si'] / adjust_sol(df['H2']) **2
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
MgO_l = LiquidSpecies("MgO")
Mg_g = GasSpecies("Mg")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l, MgO_l, Mg_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5],  MgO_l: 1, Mg_g: 1e-2}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
            ElementMassConstraint("Mg", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5],
    #           MgO_l: 1, Mg_g: 10}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue
        

filename = "SiMgOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiMgOH_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate ideal with solubility solutions

In [ ]:
filename = "SiMgOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l', 'MgO_l', 'Mg_g']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# ideal with solubility
H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann()) 
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())       
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
MgO_l = LiquidSpecies("MgO")
Mg_g = GasSpecies("Mg")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l, MgO_l, Mg_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5],  MgO_l: 1, Mg_g: 10}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
            ElementMassConstraint("Mg", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5],
    #           MgO_l: insol[6], Mg_g: insol[7]}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue


filename = "SiMgOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiMgOH_solubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate real with no solubility solutions

In [ ]:
filename = "SiMgOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l', 'MgO_l', 'Mg_g']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# real gas H2 and H2O
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"])
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"])       
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
MgO_l = LiquidSpecies("MgO")
Mg_g = GasSpecies("Mg")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l, MgO_l, Mg_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5],  MgO_l: insols[0][6], Mg_g: insols[0][7]}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
            ElementMassConstraint("Mg", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5],
    #           MgO_l: insol[6], Mg_g: insol[7]}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue


filename = "SiMgOH_nosolubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiMgOH_nosolubility_real_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate real with solubility solutions

In [ ]:
filename = "SiMgOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l', 'MgO_l', 'Mg_g']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# real gas with solubility  
H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann(), eos=get_chabrier_eos_models()["H2"])
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi(), eos=get_holland_eos_models()["H2O"])       
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
MgO_l = LiquidSpecies("MgO")
Mg_g = GasSpecies("Mg")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l, MgO_l, Mg_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5],  MgO_l: insols[0][6], Mg_g: insols[0][7]}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
            ElementMassConstraint("Mg", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5],
    #           MgO_l: insol[6], Mg_g: insol[7]}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )
        
        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiMgOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiMgOH_solubility_real_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Plot mixing ratios: ideal with no solubility + ideal with solubility + real with solubility

In [ ]:
filename = "SiMgOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
Mg_ideal = df_ideal['Mg_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiMgOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_idealsol = pd.read_csv(filename) 
ratio_idealsol = df_idealsol['H2_g'].values / df_idealsol['H2O_g'].values
logfO2_idealsol = df_idealsol['log_fO2_dIW'].values
H2_idealsol = df_idealsol['H2_g'].values
H2O_idealsol = df_idealsol['H2O_g'].values
SiH4_idealsol = df_idealsol['H4Si_g'].values
SiO_idealsol = df_idealsol['OSi_g'].values
Mg_idealsol = df_idealsol['Mg_g'].values
O2_idealsol = df_idealsol['O2_g'].values
tot_idealsol = df_idealsol['total_pressure'].values
fcH2_idealsol = df_idealsol['fugacity_coefficient_H2'].values
fcH2O_idealsol = df_idealsol['fugacity_coefficient_H2O'].values
Hmelt_idealsol = df_idealsol['melt_fraction_H'].values
log_fO2_dIWs_idealsol = df_idealsol['log_fO2_dIW'].values

filename = "SiMgOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
Mg_realsol = df_realsol['Mg_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -1.8, alpha=0.2, color='green')
ax1.axvspan(-1.8, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -5.6, alpha=0.2, color='red')
ax2.axvspan(-5.6, -0.2, alpha=0.2, color='green')
ax2.axvspan(-0.2, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -4.3, alpha=0.2, color='red')
ax3.axvspan(-4.3, -0.6, alpha=0.2, color='green')
ax3.axvspan(-0.6, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/tot_ideal, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/tot_ideal, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/tot_ideal, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/tot_ideal, color='brown', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, Mg_ideal/tot_ideal, color='cyan', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1.2])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)


ax2.plot(log_fO2_dIWs_idealsol, H2_idealsol/tot_idealsol, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, H2O_idealsol/tot_idealsol, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiH4_idealsol/tot_idealsol, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiO_idealsol/tot_idealsol, color='brown', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, Mg_idealsol/tot_idealsol, color='cyan', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'Ideal + solubility') 
ax2.set_ylim([1e-6, 1.2])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Mixing Ratios', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/tot_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/tot_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/tot_realsol, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/tot_realsol, color='brown', lw=2, ls= '-', label='SiO')
ax3.plot(log_fO2_dIWs_realsol, Mg_realsol/tot_realsol, color='cyan', lw=2, ls= '-', label='Mg')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-6, 1.2])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Mixing Ratios', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(log_fO2_dIWs_idealsol, tot_idealsol/1000, lw=2, ls= '-.', color='black', label=r'Ideal gas (solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e0, 3e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiMgOH_ideal_idealsol_realsol_%dK_%docean.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiMgOH_ideal_idealsol_realsol_%dK_%docean.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

### Plot partial pressures: ideal with no solubility + ideal with solubility + real with solubility

In [ ]:
filename = "SiMgOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
Mg_ideal = df_ideal['Mg_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
log_fO2_dIWs_ideal = df_ideal['log_fO2_dIW'].values

filename = "SiMgOH_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_idealsol = pd.read_csv(filename) 
ratio_idealsol = df_idealsol['H2_g'].values / df_idealsol['H2O_g'].values
logfO2_idealsol = df_idealsol['log_fO2_dIW'].values
H2_idealsol = df_idealsol['H2_g'].values
H2O_idealsol = df_idealsol['H2O_g'].values
SiH4_idealsol = df_idealsol['H4Si_g'].values
SiO_idealsol = df_idealsol['OSi_g'].values
Mg_idealsol = df_idealsol['Mg_g'].values
O2_idealsol = df_idealsol['O2_g'].values
tot_idealsol = df_idealsol['total_pressure'].values
fcH2_idealsol = df_idealsol['fugacity_coefficient_H2'].values
fcH2O_idealsol = df_idealsol['fugacity_coefficient_H2O'].values
Hmelt_idealsol = df_idealsol['melt_fraction_H'].values
log_fO2_dIWs_idealsol = df_idealsol['log_fO2_dIW'].values

filename = "SiMgOH_solubility_real_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
Mg_realsol = df_realsol['Mg_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
log_fO2_dIWs_realsol = df_realsol['log_fO2_dIW'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

#ax1.axvspan(-6, -5.8, alpha=0.2, color='red')
ax1.axvspan(-6, -1.8, alpha=0.2, color='green')
ax1.axvspan(-1.8, 2, alpha=0.2, color='blue')

ax2.axvspan(-6, -5.6, alpha=0.2, color='red')
ax2.axvspan(-5.6, -0.2, alpha=0.2, color='green')
ax2.axvspan(-0.2, 2, alpha=0.2, color='blue')

ax3.axvspan(-6, -4.3, alpha=0.2, color='red')
ax3.axvspan(-4.3, -0.6, alpha=0.2, color='green')
ax3.axvspan(-0.6, 2, alpha=0.2, color='blue')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(-3.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(-2.2, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(log_fO2_dIWs_ideal, H2_ideal/1e3, color='orange', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, H2O_ideal/1e3, color='blue', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiH4_ideal/1e3, color='red', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, SiO_ideal/1e3, color='brown', lw=2, ls= '--')
ax1.plot(log_fO2_dIWs_ideal, Mg_ideal/1e3, color='cyan', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-8, 2e3])
ax1.set_yscale('log')
ax1.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax1.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)


ax2.plot(log_fO2_dIWs_idealsol, H2_idealsol/1e3, color='orange', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, H2O_idealsol/1e3, color='blue', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiH4_idealsol/1e3, color='red', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, SiO_idealsol/1e3, color='brown', lw=2, ls= '-.')
ax2.plot(log_fO2_dIWs_idealsol, Mg_idealsol/1e3, color='cyan', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (H = %.1f %% planet mass, $T_{\rm surface}$ = %d K)'%(hmp,surface_temperature)) 

ax2.set_title(r'Ideal + solubility') 
ax2.set_ylim([1e-8, 2e3])
ax2.set_yscale('log')
ax2.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax2.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax3.plot(log_fO2_dIWs_realsol, H2_realsol/1e3, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(log_fO2_dIWs_realsol, H2O_realsol/1e3, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(log_fO2_dIWs_realsol, SiH4_realsol/1e3, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(log_fO2_dIWs_realsol, SiO_realsol/1e3, color='brown', lw=2, ls= '-', label='SiO')
ax3.plot(log_fO2_dIWs_realsol, Mg_realsol/1e3, color='cyan', lw=2, ls= '-', label='Mg')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-8, 2e3])
ax3.set_yscale('log')
ax3.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax3.set_ylabel(r'Partial Pressure [kbar]', fontsize=14)

ax4.plot(log_fO2_dIWs_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(log_fO2_dIWs_idealsol, tot_idealsol/1000, lw=2, ls= '-.', color='black', label=r'Ideal gas (solubility)')
ax4.plot(log_fO2_dIWs_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e0, 3e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'log$_{10}$ $f_{\rm O_2}$ ($\Delta$IW)', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiMgOH_ideal_idealsol_realsol_%dK_%docean_pressures.pdf"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.savefig("SiMgOH_ideal_idealsol_realsol_%dK_%docean_pressures.png"%(surface_temperature,massH_ocean), bbox_inches='tight')
plt.show()

## 3C. H-He-O-Si system (ideal vs real) - parameter: fO2

In [ ]:
surface_temperature=3000 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

massH_ocean_previous = 800
massH_ocean = 1000 # mass of H equivalent to water oceans
h_kg_previous: float = earth_oceans_to_kg(massH_ocean_previous)
h_kg: float = earth_oceans_to_kg(massH_ocean)

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

log_fO2_dIWs = np.linspace(-6, 2, num=200)

### Calculate ideal with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_3000K_800ocean.csv"
#filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean_previous)

df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
He_g = GasSpecies("He")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_l, He_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5], He_g: 1e2}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("He", value=h_kg/739*240),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOHHe_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOHHe_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate ideal solubility solutions

In [ ]:
filename = "SiOHHe_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l', 'He_g']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# ideal with solubility
H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann())
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
He_g = GasSpecies("He")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l, He_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5], He_g: 1e2}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []
i = 0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("He", value=h_kg/739*240),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue


filename = "SiOHHe_solubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOHHe_solubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

## 3D. H-He-O-C-N system (ideal vs real) - parameter: fO2

### Calculate ideal with no solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_3000K_800ocean.csv"
#filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean_previous)

df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
CO2_g = GasSpecies("CO2")
CO_g = GasSpecies("CO")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_l, CO2_g, CO_g, CH4_g, N2_g, NH3_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5], CO2_g: 1e2, CO_g: 1e2, CH4_g: 1e-2,
          N2_g: 1e2, NH3_g: 1e4}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("C", value=h_kg*4.6/739),
            ElementMassConstraint("N", value=h_kg*0.96/739),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOHCN_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOHCN_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

### Calculate ideal solubility solutions

In [ ]:
filename = "SiOHCN_nosolubility_ideal_3000K_800ocean.csv"
#filename = "SiOH_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean_previous)

df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")
CO2_g = GasSpecies("CO2")
CO_g = GasSpecies("CO")
CH4_g = GasSpecies("CH4")
N2_g = GasSpecies("N2")
NH3_g = GasSpecies("NH3")

species = Species([H2_g, H2O_g, O2_g, H4Si_g, OSi_g, O2Si_l, CO2_g, CO_g, CH4_g, N2_g, NH3_g])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], 
          H4Si_g: insols[0][4], O2Si_l: insols[0][5], CO2_g: insols[0][6], CO_g: insols[0][7], 
          CH4_g: insols[0][8], N2_g: insols[0][9], NH3_g: insols[0][10]}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for log_fO2_dIW in log_fO2_dIWs:

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("C", value=h_kg*4.6/739),
            ElementMassConstraint("N", value=h_kg*0.96/739),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    #initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOHCN_nosolubility_ideal_%dK_%docean.csv"%(surface_temperature,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOHCN_nosolubility_ideal_%dK_%docean"%(surface_temperature,massH_ocean), to_excel=True, to_pickle=False)

## 4. H-O-Si system (ideal vs real) - parameter: amount of H 

### Set basic parameters

In [ ]:
surface_temperature=3000 # kelvin
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

log_fO2_dIW = 0

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

massH_oceans = np.logspace(0, 4, num=200)

### Calculate ideal with no solubility solutions

In [ ]:
planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: 1e5, H2O_g: 1e0, O2_g: 1e-10, OSi_g: 1e2, H4Si_g: 1e4, O2Si_l: 1e0}
#initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for massH_ocean in massH_oceans:

    h_kg: float = earth_oceans_to_kg(massH_ocean)

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            #TotalPressureConstraint(value=total_pressure),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, max_attempts=50, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try: 
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['hmp'] = h_kg / planet_mass * 100
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_ideal_%dK_IW%dlogfO2"%(surface_temperature,log_fO2_dIW), to_excel=True, to_pickle=False)

### Calculate real with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Real gas
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"])
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"])        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for massH_ocean in massH_oceans:

    h_kg: float = earth_oceans_to_kg(massH_ocean)   

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            #TotalPressureConstraint(value=total_pressure),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['hmp'] = h_kg / planet_mass * 100
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_real_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_real_%dK_IW%dlogfO2"%(surface_temperature,log_fO2_dIW), to_excel=True, to_pickle=False)

### Calculate ideal with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)


H2_g = GasSpecies("H2", solubility=H2_basalt_hirschmann())
H2O_g = GasSpecies("H2O", solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for massH_ocean in massH_oceans:

    h_kg: float = earth_oceans_to_kg(massH_ocean)   

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            #TotalPressureConstraint(value=total_pressure),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    # initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    # initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['hmp'] = h_kg / planet_mass * 100
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_solubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_solubility_ideal_%dK_IW%dlogfO2"%(surface_temperature,log_fO2_dIW), to_excel=True, to_pickle=False)

### Calculate real with solubility solutions

In [ ]:
filename = "SiOH_solubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius)

# Non-ideal H2
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"], solubility=H2_basalt_hirschmann())
#### Re-defining remaining species 
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"], solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

system = InteriorAtmosphereSystem(species=species, planet=planet)

solutions = []

i=0
for massH_ocean in massH_oceans:

    h_kg: float = earth_oceans_to_kg(massH_ocean)   

    constraint: SystemConstraints = SystemConstraints(
        [
            BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
            #TotalPressureConstraint(value=total_pressure),
            ElementMassConstraint("H", value=h_kg),
            ElementMassConstraint("Si", value=si_kg),
        ]
    )

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    # initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    # initial_solution_first = initial_solution
    
    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['hmp'] = h_kg / planet_mass * 100
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_solubility_real_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_solubility_real_%dK_IW%dlogfO2"%(surface_temperature,log_fO2_dIW), to_excel=True, to_pickle=False)


### Plot ideal with no solubility vs ideal with solubility vs real with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
hmp_ideal = df_ideal['hmp'].values

filename = "SiOH_solubility_ideal_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df_idealsol = pd.read_csv(filename) 
ratio_idealsol = df_idealsol['H2_g'].values / df_idealsol['H2O_g'].values
logfO2_idealsol = df_idealsol['log_fO2_dIW'].values
H2_idealsol = df_idealsol['H2_g'].values
H2O_idealsol = df_idealsol['H2O_g'].values
SiH4_idealsol = df_idealsol['H4Si_g'].values
SiO_idealsol = df_idealsol['OSi_g'].values
O2_idealsol = df_idealsol['O2_g'].values
tot_idealsol = df_idealsol['total_pressure'].values
fcH2_idealsol = df_idealsol['fugacity_coefficient_H2'].values
fcH2O_idealsol = df_idealsol['fugacity_coefficient_H2O'].values
Hmelt_idealsol = df_idealsol['melt_fraction_H'].values
hmp_idealsol = df_idealsol['hmp'].values

filename = "SiOH_solubility_real_%dK_IW%dlogfO2.csv"%(surface_temperature,log_fO2_dIW)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
hmp_realsol = df_realsol['hmp'].values

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

ax1.axvspan(1e-4, 1e1, alpha=0.2, color='green')

ax2.axvspan(1e-4, 2e-2, alpha=0.2, color='red')
ax2.axvspan(2e-2, 1e1, alpha=0.2, color='green')

ax3.axvspan(1e-4, 2e-2, alpha=0.2, color='red')
ax3.axvspan(2e-2, 1e1, alpha=0.2, color='green')

#ax2 = ax1.twinx()

#ax1.text(-5.8, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax1.text(1e-1, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')


ax2.text(1e-3, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(1e-1, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')
# ax2.text(1.2, 1e-5, 'Steam \nWorlds', fontsize=10, horizontalalignment='center')

ax1.plot(hmp_ideal, H2_ideal/tot_ideal, color='orange', lw=2, ls= '--')
ax1.plot(hmp_ideal, H2O_ideal/tot_ideal, color='blue', lw=2, ls= '--')
ax1.plot(hmp_ideal, SiH4_ideal/tot_ideal, color='red', lw=2, ls= '--')
ax1.plot(hmp_ideal, SiO_ideal/tot_ideal, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1.2])
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'Hydrogen mass fraction [%]', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)


ax2.plot(hmp_idealsol, H2_idealsol/tot_idealsol, color='orange', lw=2, ls= '-.')
ax2.plot(hmp_idealsol, H2O_idealsol/tot_idealsol, color='blue', lw=2, ls= '-.')
ax2.plot(hmp_idealsol, SiH4_idealsol/tot_idealsol, color='red', lw=2, ls= '-.')
ax2.plot(hmp_idealsol, SiO_idealsol/tot_idealsol, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (IW %.1f log fO2, $T_{\rm surface}$ = %d K)'%(log_fO2_dIW,surface_temperature)) 

ax2.set_title(r'Ideal + solubility') 
ax2.set_ylim([1e-6, 1.2])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel(r'Hydrogen mass fraction [%]', fontsize=14)
ax2.set_ylabel(r'Mixing Ratios', fontsize=14)

ax3.plot(hmp_realsol, H2_realsol/tot_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(hmp_realsol, H2O_realsol/tot_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(hmp_realsol, SiH4_realsol/tot_realsol, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(hmp_realsol, SiO_realsol/tot_realsol, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-6, 1.2])
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel(r'Hydrogen mass fraction [%]', fontsize=14)
ax3.set_ylabel(r'Mixing Ratios', fontsize=14)

ax4.plot(hmp_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(hmp_idealsol, tot_idealsol/1000, lw=2, ls= '-.', color='black', label=r'Ideal gas (solubility)')
ax4.plot(hmp_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e-2, 3e4])
ax4.set_xscale('log')
ax4.set_yscale('log')
ax4.set_xlabel(r'Hydrogen mass fraction [%]', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_idealsol_realsol_%dK_IW%dlogfO2.pdf"%(surface_temperature,log_fO2_dIW), bbox_inches='tight')
plt.savefig("SiOH_ideal_idealsol_realsol_%dK_IW%dlogfO2.png"%(surface_temperature,log_fO2_dIW), bbox_inches='tight')
plt.show()

## 5. H-O-Si system (ideal vs real) - parameter: Surface Temperature

### Set basic parameters

In [ ]:
planet_mass=4.6*5.972e24 # kg
surface_radius=1.5*6371000 # metre

log_fO2_dIW = 0

si_kg: float = 0.1459 * planet_mass # Si = 14.59 wt% Kargel & Lewis (1993)  

massH_ocean = 1200
h_kg: float = earth_oceans_to_kg(massH_ocean)

surface_temperatures = np.linspace(2000, 4500, num=200)

### Calculate ideal with no solubility solutions

In [ ]:
# Ideal gases (no solubility)
H2_g = GasSpecies("H2")
H2O_g = GasSpecies("H2O")        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: 1e5, H2O_g: 1e0, O2_g: 1e-10, OSi_g: 1e2, H4Si_g: 1e4, O2Si_l: 1e0}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

constraint: SystemConstraints = SystemConstraints(
    [
        BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
        ElementMassConstraint("H", value=h_kg),
        ElementMassConstraint("Si", value=si_kg),
    ]
)

system = InteriorAtmosphereSystem(species=species)

solutions = []

i=0
for surface_temperature in surface_temperatures:

    planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius) 

    setattr(system, "planet", planet)

    # initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    # initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['surface_temperature'] = surface_temperature
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_ideal_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_ideal_IW%dlogfO2_%docean"%(log_fO2_dIW,massH_ocean), to_excel=True, to_pickle=False)

### Calculate real with no solubility solutions

In [ ]:
filename = "SiOH_nosolubility_ideal_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

# Non-ideal H2 and H2O
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"])
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"])        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

constraint: SystemConstraints = SystemConstraints(
    [
        BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
        ElementMassConstraint("H", value=h_kg),
        ElementMassConstraint("Si", value=si_kg),
    ]
)

system = InteriorAtmosphereSystem(species=species)

solutions = []

i=0
for surface_temperature, insol in zip(surface_temperatures, insols):

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius) 

    setattr(system, "planet", planet)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['surface_temperature'] = surface_temperature
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_nosolubility_real_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_nosolubility_real_IW%dlogfO2_%docean"%(log_fO2_dIW,massH_ocean), to_excel=True, to_pickle=False)

### Calcualte real solutions with solubility

In [ ]:
filename = "SiOH_nosolubility_ideal_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df = pd.read_csv(filename) 
df = df[['H2_g', 'H2O_g', 'O2_g', 'OSi_g', 'H4Si_g', 'O2Si_l']]
insols = df.to_numpy()

# Non-ideal H2 and H2O with solubility
H2_g = GasSpecies("H2", eos=get_chabrier_eos_models()["H2"], solubility=H2_basalt_hirschmann())
H2O_g = GasSpecies("H2O", eos=get_holland_eos_models()["H2O"], solubility=H2O_peridotite_sossi())        
O2_g = GasSpecies("O2")
OSi_g = GasSpecies("OSi")
H4Si_g = GasSpecies("H4Si")
O2Si_l = LiquidSpecies("O2Si")

species = Species([H2_g, H2O_g, O2_g, OSi_g, H4Si_g, O2Si_l])

first_solution = {H2_g: insols[0][0], H2O_g: insols[0][1], O2_g: insols[0][2], OSi_g: insols[0][3], H4Si_g: insols[0][4], O2Si_l: insols[0][5]}
# initial_solution_first = InitialSolutionDict(values, species=species)

initial_solution = InitialSolutionLast(InitialSolutionDict(first_solution, species=species), species=species)

constraint: SystemConstraints = SystemConstraints(
    [
        BufferedFugacityConstraint(O2_g, IronWustiteBuffer(log10_shift=log_fO2_dIW)),
        ElementMassConstraint("H", value=h_kg),
        ElementMassConstraint("Si", value=si_kg),
    ]
)

system = InteriorAtmosphereSystem(species=species)

solutions = []

i=0
for surface_temperature, insol in zip(surface_temperatures, insols):

    # values = {H2_g: insol[0], H2O_g: insol[1], O2_g: insol[2], OSi_g: insol[3], H4Si_g: insol[4], O2Si_l: insol[5]}

    # initial_solution = InitialSolutionDict(values, species=species)

    planet = Planet(surface_temperature=surface_temperature, planet_mass=planet_mass, 
                  surface_radius=surface_radius) 

    setattr(system, "planet", planet)

    #initial_solution = InitialSolutionLast(initial_solution_first, species=species)
    system.solve(constraint, initial_solution=initial_solution, factor=0.1)
    #initial_solution_first = initial_solution

    try:  
        solution = system.solution.solution_dict()
        solution['total_pressure'] = system.atmosphere_pressure
        solution['log_fO2_dIW'] = log_fO2_dIW
        solution['surface_temperature'] = surface_temperature
        solution['melt_fraction_H'] = 2.016e-3 / h_kg * (
            system.output['H2_g'][i]['melt_moles']+system.output['H2O_g'][i]['melt_moles']
            )

        solution['fugacity_coefficient_H2'] = system.output['H2_g'][i]['fugacity_coefficient']
        solution['fugacity_coefficient_H2O'] = system.output['H2O_g'][i]['fugacity_coefficient']
        solutions.append(solution)
        i = i + 1
    except IndexError:
        continue

filename = "SiOH_solubility_real_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

system.failed_solves

data = system.output("SiOH_solubility_real_IW%dlogfO2_%docean"%(log_fO2_dIW,massH_ocean), to_excel=True, to_pickle=False)

### Plot mixing ratios

In [ ]:
filename ="SiOH_nosolubility_ideal_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df_ideal = pd.read_csv(filename) 
ratio_ideal = df_ideal['H2_g'].values / df_ideal['H2O_g'].values
logfO2_ideal = df_ideal['log_fO2_dIW'].values
H2_ideal = df_ideal['H2_g'].values
H2O_ideal = df_ideal['H2O_g'].values
SiH4_ideal = df_ideal['H4Si_g'].values
SiO_ideal = df_ideal['OSi_g'].values
O2_ideal = df_ideal['O2_g'].values
tot_ideal = df_ideal['total_pressure'].values
fcH2_ideal = df_ideal['fugacity_coefficient_H2'].values
fcH2O_ideal = df_ideal['fugacity_coefficient_H2O'].values
Hmelt_ideal = df_ideal['melt_fraction_H'].values
Tsurf_ideal = df_ideal['surface_temperature'].values

filename = "SiOH_nosolubility_real_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df_real = pd.read_csv(filename) 
ratio_real = df_real['H2_g'].values / df_real['H2O_g'].values
logfO2_real = df_real['log_fO2_dIW'].values
H2_real = df_real['H2_g'].values
H2O_real = df_real['H2O_g'].values
SiH4_real = df_real['H4Si_g'].values
SiO_real = df_real['OSi_g'].values
O2_real = df_real['O2_g'].values
tot_real = df_real['total_pressure'].values
fcH2_real = df_real['fugacity_coefficient_H2'].values
fcH2O_real = df_real['fugacity_coefficient_H2O'].values
Hmelt_real = df_real['melt_fraction_H'].values
Tsurf_real = df_real['surface_temperature'].values

filename = "SiOH_solubility_real_IW%dlogfO2_%docean.csv"%(log_fO2_dIW,massH_ocean)
df_realsol = pd.read_csv(filename) 
ratio_realsol = df_realsol['H2_g'].values / df_realsol['H2O_g'].values
logfO2_realsol = df_realsol['log_fO2_dIW'].values
H2_realsol = df_realsol['H2_g'].values
H2O_realsol = df_realsol['H2O_g'].values
SiH4_realsol = df_realsol['H4Si_g'].values
SiO_realsol = df_realsol['OSi_g'].values
O2_realsol = df_realsol['O2_g'].values
tot_realsol = df_realsol['total_pressure'].values
fcH2_realsol = df_realsol['fugacity_coefficient_H2'].values
fcH2O_realsol = df_realsol['fugacity_coefficient_H2O'].values
Hmelt_realsol = df_realsol['melt_fraction_H'].values
Tsurf_realsol = df_realsol['surface_temperature'].values

hmp = h_kg / planet_mass * 100

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(6,12), tight_layout='True')

ax1.axvspan(2000, 4500, alpha=0.2, color='green')

ax2.axvspan(2000, 3000, alpha=0.2, color='red')
ax2.axvspan(3000, 4500, alpha=0.2, color='green')

ax3.axvspan(2000, 3580, alpha=0.2, color='red')
ax3.axvspan(3580, 4500, alpha=0.2, color='green')

#ax2 = ax1.twinx()

ax1.text(3000, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')

ax2.text(2500, 1e-5, 'Silicon \nWorlds', fontsize=10, horizontalalignment='center')
ax2.text(3500, 1e-5, 'Hydrogen \nWorlds', fontsize=10, horizontalalignment='center')


ax1.plot(Tsurf_ideal, H2_ideal/tot_ideal, color='orange', lw=2, ls= '--')
ax1.plot(Tsurf_ideal, H2O_ideal/tot_ideal, color='blue', lw=2, ls= '--')
ax1.plot(Tsurf_ideal, SiH4_ideal/tot_ideal, color='red', lw=2, ls= '--')
ax1.plot(Tsurf_ideal, SiO_ideal/tot_ideal, color='brown', lw=2, ls= '--')

ax1.set_title(r'Ideal + no solubility')
ax1.set_ylim([1e-6, 1.2])
ax1.set_yscale('log')
ax1.set_xlabel(r'$T_{\rm surface}$ [K]', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)


ax2.plot(Tsurf_real, H2_real/tot_real, color='orange', lw=2, ls= '-.')
ax2.plot(Tsurf_real, H2O_real/tot_real, color='blue', lw=2, ls= '-.')
ax2.plot(Tsurf_real, SiH4_real/tot_real, color='red', lw=2, ls= '-.')
ax2.plot(Tsurf_real, SiO_real/tot_real, color='brown', lw=2, ls= '-.')

fig.suptitle(r'Surface Atm. Comp. (H = %.1f %% planet mass, IW %d log fO2)'%(hmp,log_fO2_dIW)) 

ax2.set_title(r'Real + no solubility') 
ax2.set_ylim([1e-6, 1.2])
ax2.set_yscale('log')
ax2.set_xlabel(r'$T_{\rm surface}$ [K]', fontsize=14)
ax2.set_ylabel(r'Mixing Ratios', fontsize=14)

ax3.plot(Tsurf_realsol, H2_realsol/tot_realsol, color='orange', lw=2, ls= '-', label='H$_2$')
ax3.plot(Tsurf_realsol, H2O_realsol/tot_realsol, color='blue', lw=2, ls= '-', label='H$_2$O')
ax3.plot(Tsurf_realsol, SiH4_realsol/tot_realsol, color='red', lw=2, ls= '-', label='SiH$_4$')
ax3.plot(Tsurf_realsol, SiO_realsol/tot_realsol, color='brown', lw=2, ls= '-', label='SiO')

ax3.set_title(r'Real + solubility') 
ax3.set_ylim([1e-6, 1.2])
ax3.set_yscale('log')
ax3.set_xlabel(r'$T_{\rm surface}$ [K]', fontsize=14)
ax3.set_ylabel(r'Mixing Ratios', fontsize=14)

ax4.plot(Tsurf_ideal, tot_ideal/1000, lw=2, ls= '--', color='black', label=r'Ideal gas (no solubility)')
ax4.plot(Tsurf_real, tot_real/1000, lw=2, ls= '-.', color='black', label=r'Real gas (no solubility)')
ax4.plot(Tsurf_realsol, tot_realsol/1000, lw=2, ls= '-', color='black', label=r'Real gas (solubility)')

ax4.set_ylim([1e0, 3e3])
ax4.set_yscale('log')
ax4.set_xlabel(r'$T_{\rm surface}$ [K]', fontsize=14)
ax4.set_ylabel(r"$P_{\rm surface}$ [kbar]",  fontsize=14)

ax4.set_title(r'Surface Pressure')

ax3.legend(ncol=2, loc='lower left')
ax4.legend(ncol=1, loc='lower left')

plt.savefig("SiOH_ideal_real_realsol_IW%dlogfO2_%docean.pdf"%(log_fO2_dIW,massH_ocean), bbox_inches='tight')
plt.savefig("SiOH_ideal_real_realsol_IW%dlogfO2_%docean.png"%(log_fO2_dIW,massH_ocean), bbox_inches='tight')
plt.show()